In [34]:
# train.py
from torch.utils.data import Dataset, DataLoader
import torch
from model import ColBERT
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
import pytorch_lightning as pl
from transformers import BertTokenizer
import csv
import os

# model.py
import torch
from transformers import BertModel, get_linear_schedule_with_warmup
import pytorch_lightning as pl
import numpy as np

import train
import model
from transformers import AutoTokenizer, AutoModel
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
colbert = ColBERT(100, 16)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
query = 'walgreens store sales average'
passages = ['The average Walgreens salary ranges from approximately $15,000 per year for Customer Service Associate / Cashier to $179,900 per year for District Manager. Average Walgreens hourly pay ranges from approximately $7.35 per hour for Laboratory Technician to $68.90 per hour for Pharmacy Manager. Salary information comes from 7,810 data points collected directly from employees, users, and jobs on Indeed.',
 'The average revenue in 2011 of a Starbuck Store was $1,078,000, up  from $1,011,000 in 2010.    The average ticket (total purchase) at domestic Starbuck stores in  No … vember 2007 was reported at $6.36.    In 2008, the average ticket was flat (0.0% change).',
 'In fiscal 2014, Walgreens opened a total of 184 new locations and acquired 84 locations, for a net decrease of 273 after relocations and closings. How big are your stores? The average size for a typical Walgreens is about 14,500 square feet and the sales floor averages about 11,000 square feet. How do we select locations for new stores? There are several factors that Walgreens takes into account, such as major intersections, traffic patterns, demographics and locations near hospitals.',
 'th store in 1984, reaching $4 billion in sales in 1987, and $5 billion two years later. Walgreens ended the 1980s with 1,484 stores, $5.3 billion in revenues and $154 million in profits. However, profit margins remained just below 3 percent of sales, and returns on assets of less than 10 percent.',
 'The number of Walgreen stores has risen from 5,000 in 2005 to more than 8,000 at present. The average square footage per store stood at approximately 10,200 and we forecast the figure to remain constant over our review period. Walgreen earned $303 as average front-end revenue per store square foot in 2012.',
 'Your Walgreens Store. Select a store from the search results to make it Your Walgreens Store and save time getting what you need. Your Walgreens Store will be the default location for picking up prescriptions, photos, in store orders and finding deals in the Weekly Ad.']

query = tokenizer(query,
                 padding = 'longest',
                 truncation = True,
                 max_length = 512)

query_input_ids = torch.tensor([query['input_ids']])
query_attention_masks = torch.tensor([query['attention_mask']])

passages = tokenizer(passages,
                     padding = 'longest',
                     truncation = True,
                     max_length = 512)

doc_input_ids = torch.tensor(passages['input_ids'])
doc_attention_masks = torch.tensor(passages['attention_mask'])

D = model(doc_input_ids, doc_attention_masks)[0]
Q = model(query_input_ids, query_attention_masks)[0]

In [32]:
x = {
        'query_input_ids': query_input_ids,
        'query_attention_mask': query_attention_masks,
        'positive_input_ids': doc_input_ids,
        'positive_attention_mask': doc_attention_masks,
        'negative_input_ids': doc_attention_masks,
        'negative_attention_mask': doc_attention_masks
    }

In [35]:
colbert.forward(x)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/dminn/Projects/msmarco/msmarco_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_291/1244610806.py", line 1, in <cell line: 1>
    colbert.forward(x)
  File "/home/dminn/Projects/passage-retrieval-with-noise/colbert/model.py", line 54, in forward
    pos_scores = late_interaction(query_vectors, positive_vectors) # shape: (B, B)
NameError: name 'late_interaction' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dminn/Projects/msmarco/msmarco_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 1982, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/home/dminn/Projects/msmarco/msmarco_env/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1118, in structured_traceback
    return Formatted